<a href="https://colab.research.google.com/github/qsmr/GH_collab/blob/main/used_car_price_prediction_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **USED CAR PRICE PREDICTION**

**`Procedere con l'esecuzione di ogni singola cella aspettando che termini la computazione.`**

In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

### Download file for plotting complexity curves

In [2]:
!curl https://raw.githubusercontent.com/francescopisu/Used-car-price-prediction/master/curves.py --output curves.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  3760  100  3760    0     0   7163      0 --:--:-- --:--:-- --:--:--  7161


### Downlod dataset

In [3]:
# May take up to 5 minutes depending on connection
!curl https://raw.githubusercontent.com/francescopisu/Used-car-price-prediction/master/data/cars.csv --output cars.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 86.2M  100 86.2M    0     0  12.6M      0  0:00:06  0:00:06 --:--:-- 23.8M


In [4]:
!pip install spark-sklearn
!pip install pyspark

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 18.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark-sklearn: filename=spark_sklearn-0.3.0-py3-none-any.whl size=30577 sha256=db2369f412037eeff4fb12fe2189b8eb4d0091d01cba83681669ed896f74297d
  Stored in directory: /root/.cache/pip/wheels/83/c3/49/1d965f070cae453ee0192dc86655721447970d1572377f5e3a
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for scikit-learn
  Running setup.py clean for scikit-learn
Successfully built spark-sklearn
Failed to build scikit-learn
ERROR: Could not build wheels for scikit-learn, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.6 MB/s eta 0:

In [5]:
pip install rfpimp

  Preparing metadata (setup.py) ... done
  Created wheel for rfpimp: filename=rfpimp-1.3.7-py3-none-any.whl size=10649 sha256=5aec06078f6b93d33e390a8888eba0402ed38104d2e5ac6b2534aa8c023ab348
  Stored in directory: /root/.cache/pip/wheels/6a/12/08/d5bc35127c8d69d39c1f3736a95419ab4763cc0c80ed65bf41
Successfully built rfpimp


**IMPORTING LIBRARIES**

In [6]:
import pandas as pd
import seaborn as sns
import numpy as np
import datetime
import matplotlib
import numpy as np
import pickle
import matplotlib.pyplot as plt
import curves
from sklearn import preprocessing
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer
from spark_sklearn import GridSearchCV
from spark_sklearn.util import createLocalSparkSession
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

ModuleNotFoundError: ignored

**READ THE CSV INPUT FILE**

In [ ]:
# Read data
cars = pd.read_csv('cars.csv', encoding='latin1', error_bad_lines=False, warn_bad_lines=False)

In [ ]:
cars.sample(5)

In [ ]:
cars.columns

## ANALYSING PRICE ATTRIBUTE

In [ ]:
cars['Price'].describe()

In [ ]:
sns.distplot(cars['Price'])
#skewness and kurtosis
print("Skewness: %f" % cars['Price'].skew())
print("Kurtosis: %f" % cars['Price'].kurt())

"""
We can observe that the distribution of prices shows a high positive skewness
to the left (skew > 1). A kurtosis value of 60 is extremely high, meaning that
there is a profusion of outliers in the dataset. We need to do something.
"""

### RELATIONSHIP WITH NUMERICAL FEATURES

In [ ]:
# Scatter plot Mileage / Price
attrib = 'Mileage'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
data.plot.scatter(x=attrib, y='Price', ylim=(0,450000));

"""
We see that the lesser is the mileage, the higher tends to be the price.
Price and Mileage seem to be in an exponential relationship with negative
exponent. It's a typical form of exponential decay.
"""

In [ ]:
# Scatter plot Year / Price
attrib = 'Year'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
data.plot.scatter(x=attrib, y='Price', ylim=(0,450000));

"""
Prices tend to be higher as cars are more recent, and viceversa.
"""

### RELATIONSHIP WITH CATEGORICAL FEATURES

In [ ]:
# Box plot Make (Car manufacturers) / Price

attrib = 'Make'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
f, ax = plt.subplots(figsize=(20, 12))
fig = sns.boxplot(x=attrib, y="Price", data=data)
fig.axis(ymin=0, ymax=450000);

"""
We can observe there is a correlation between Price and Exotic/Luxury car manufacturers.
On a sidenote, there is a prevalence of low to medium budget cars in the dataset

Several car manufacturers as Porsche, Audi, Chevreolet ecc.. have a strong
presence of outliers, altought outliers may not be the right term because
those makers produce car models that cost more than others.
"""

In [ ]:
# Box plot Year / Price

attrib = 'Year'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
f, ax = plt.subplots(figsize=(18, 12))
fig = sns.boxplot(x=attrib, y="Price", data=data)
fig.axis(ymin=0, ymax=450000);

"""
We can observe a slight increase of Prices the most recent years. Not a strong
tendency though.
"""

In [ ]:
# Box plot State / Price

attrib = 'State'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
f, ax = plt.subplots(figsize=(18, 12))
fig = sns.boxplot(x=attrib, y="Price", data=data)
fig.axis(ymin=0, ymax=450000);

### FEATURE IMPORTANCE RELATED TO TARGET


In [ ]:
# Find most important features relative to target Price
print("Find most important features relative to target")
corr = cars.corr()
corr.sort_values(["Price"], ascending = False, inplace = True)
print(corr.Price)

"""
Between Year and Price there is a positive correlation, meaning that the higher
is the Year (more recent), the higher is the Price (more recent cars cost more).
Between Price and Mileage there is a negative correlation, meaning that higher
is the mileage, lower is the Price (cars with high mileage cost less).

We obviously will get rid of Id attribute.
"""

### CORRELATION MATRIX

In [ ]:
# General correlation matrix
corrmat = cars.corr()
f, ax = plt.subplots(figsize=(7, 7))
sns.heatmap(corrmat, vmax=.8, square=True);

"""
There is a mild positive correlation between Year and Price and a mild
negative correlation between Year and Mileage (as we have already said).
Moreover the is a strong negative correlation between Mileage and Year,
meaning that the older is the car, the higher will be its mileage.
Obviously there are exceptions but that is the general trend.
"""

In [ ]:
# Price correlation matrix
k = 4 #number of variables for heatmap
cols = corrmat.nlargest(k, 'Price')['Price'].index
cm = np.corrcoef(cars[cols].values.T)
sns.set(font_scale=1.25)
hm = sns.heatmap(cm, cbar=True, annot=True, square=True, fmt='.2f', annot_kws={'size': 10}, yticklabels=cols.values, xticklabels=cols.values)
plt.show()

"""
This plot shows what we said earies but in form of numbers.
"""

### SCATTERPLOTS

In [ ]:
# Scatter plot for numerical features
num_features = ["Price", "Mileage", "Year"]
sns.pairplot(cars[num_features], size = 2.5)
plt.show();

## **PREPROCESSING**

In [ ]:
# Checking for missing values -> no missing values
cars.isnull().sum()

In [ ]:
# Checking for plausible values of numerical features -> we can see that the maximum value of mileage is 77milion km
# we need to fix these problems
cars.describe()

In [ ]:
# Record distribution over the year
sns.distplot((cars["Year"]))

In [ ]:
# Checking values of categorical features
cat_val = ["City", "State", "Vin", "Make","Model"]
for col in cat_val:
    print ([col]," : ",cars[col].unique())

# Features like Id, State and Vin can be discarded because wouldn't bring anything good to our purpose

### OUTLIERS MANAGEMENT

**BIVARIATE ANALYSIS**

In [ ]:
# Bivariate analysis Price / Year
attrib = 'Year'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
data.plot.scatter(x=attrib, y='Price', ylim=(0,420000));

**REMOVING OUTLIERS BY MODEL**

In [ ]:
# Box Plot showing car manufacturer / Price range

attrib = 'Make'
data = pd.concat([cars['Price'], cars[attrib]], axis=1)
f, ax = plt.subplots(figsize=(20, 12))
fig = sns.boxplot(x=attrib, y="Price", data=data)
fig.axis(ymin=0, ymax=450000);

"""
We can see a certain number of outliers. Now we'll try to get rid of those.
"""

gender, tissue_type, tissue_value
F, 'cerebrum_l_wm', 10
M, 'cerebrum_r_gm', 20
F, 'cerebrum_r_wm', 10
M, 'cerebrum_r_gm', 20
F, 'cerebrum_l_gm', 10
M, 'cerebrum_l_gm', 20
F, 'cerebrum_r_wm', 10
M, 'cerebrum_r_gm', 20

boxplot(x="tissue_type", y="tissue_value", hue="gender")

In [ ]:
from numpy import percentile
from functools import partial

category='Model'

models = cars[category].unique()

"""
Here we are taking the values between the 25th and 80th percentile of the
gaussian curve.
"""
p25 = cars.groupby(by=category)[category,'Price'].agg(lambda x: percentile(x['Price'], 20))
p80 = cars.groupby(by=category)[category,'Price'].agg(lambda x: percentile(x['Price'], 80))

In [ ]:
# Now this strategy is applied to the 3k models; it will take up to 10 minutes
# depending on the hardware
models = cars[category].unique()

cars_cleaned = pd.DataFrame()

for i,m in enumerate(models):
    cars_cleaned=cars_cleaned.append(cars[( (cars[category] == m ) & ( (cars["Price"] > p25.Model[m]) & (cars["Price"] <= p80.Model[m])) ) ],ignore_index=True)


In [ ]:
cars_cleaned.to_csv('cars_outliers_removed.csv')

### ATTENTION
Alternatively, the cleaned_dataset can be downloaded executing the cell above:

In [ ]:
!curl https://raw.githubusercontent.com/francescopisu/Used-car-price-prediction/master/data/cars_outliers_removed.csv --output cars_outliers_removed.csv

In [ ]:
# Reading the newly cleaned dataset
cleaned_cars = pd.read_csv("cars_outliers_removed.csv", encoding='latin1', error_bad_lines=False,warn_bad_lines=False)

In [ ]:
# Box Plot showing car manufacturer / Price range after cleaning

attrib = 'Make'
data = pd.concat([cleaned_cars['Price'], cleaned_cars[attrib]], axis=1)
f, ax = plt.subplots(figsize=(20, 12))
fig = sns.boxplot(x=attrib, y="Price", data=data)
fig.axis(ymin=0, ymax=450000);

"""
The outliers have been eliminated.
"""

In [ ]:
print("Too new: %d" % cleaned_cars.loc[cleaned_cars.Year >= 2017].count()['Id'])
print("Too few km: " , cleaned_cars.loc[cleaned_cars.Mileage < 5000].count()['Id'])
print("Too many km: " , cleaned_cars.loc[cleaned_cars.Mileage > 250000].count()['Id'])

In [ ]:
# Drop some unuseful columns with respect to our analysis
cleaned_cars = cleaned_cars.drop(["Id", "State", "Vin", "City"], axis=1)

# Replace the NaN values for categoric attributes
cleaned_cars['Make'].fillna(value='blank', inplace=True)
cleaned_cars['Model'].fillna(value='blank', inplace=True)


# Drop duplicates
cleaned_cars = cleaned_cars.drop_duplicates(["Year", "Mileage", "Price", "Make", "Model"])

# Remove outliers
cleaned_cars = cleaned_cars[
        (cleaned_cars.Year <= 2017)
      & (cleaned_cars.Year >= 2008)
      & (cleaned_cars.Mileage >= 5000)
      & (cleaned_cars.Mileage <= 250000)]

# Remove extra column
cleaned_cars = cleaned_cars.drop(["Unnamed: 0"], axis=1)

In [ ]:
# Scatter plot for numerical features
num_features = ["Price", "Mileage", "Year"]
sns.pairplot(cleaned_cars[num_features], size = 2.5)
plt.show();

# In the Mileage/Price plot we observe that the lesser is the mileage, the higher
# is the price. In the Year/Price plot instead, we observe that the prices
# are somewhat equally distributed between the various years.

In [ ]:
cleaned_cars.sample(5)

### TOWARDS NORMAL DISTRIBUTION OF PRICES


In [ ]:
# Distribution of prices
from scipy.stats import norm
from scipy import stats

# Using Histogram and normal probability plot
sns.distplot(cleaned_cars['Price'], fit=norm);
fig = plt.figure()
res = stats.probplot(cleaned_cars['Price'], plot=plt)

In [ ]:
#applying log transformation
cleaned_cars['Price'] = np.log(cleaned_cars['Price'])

In [ ]:
#transformed histogram and normal probability plot
sns.distplot(cleaned_cars['Price'], fit=norm);
fig = plt.figure()
res = stats.probplot(cleaned_cars['Price'], plot=plt)

### LABEL ENCODING

In [ ]:
# LABEL ENCODING
features = ['Make', 'Model']
les = {}

for f in features:
  les[f] = preprocessing.LabelEncoder()
  les[f] = les[f].fit(cleaned_cars[f])
  cleaned_cars[f] = les[f].transform(cleaned_cars[f])

### TRAIN / TEST SPLIT

In [ ]:
# Splitting the dataset into train and test sets
train_set, test_set = train_test_split(cleaned_cars, test_size = 0.33, random_state = 42)

"""
The Test Set in this case is our Hold-out set that we'll be using later
for final validation
"""

# Separating target labels from the rest
cars_train = train_set.drop("Price", axis=1) #train without target
cars_price_train = train_set["Price"].copy() #target

cars_test  = test_set.drop("Price", axis=1)
cars_price_test = test_set["Price"].copy()


## **TRAINING AND COMPARING MODELS**

**BEST SCORE FUNCTION**

In [ ]:
# This function returns the best score achieved by the model over all the cv splits
def best_score(forest, cv):
  best_score = 0
  for i in range(0, cv):
    items = list(map(lambda x: abs(x), forest.cv_results_['split'+str(i)+'_test_score']))
    arr = np.append(best_score, items)
    best_score = max(arr)

  return best_score

**BEST PARAMS FUNCTION**

In [ ]:
# This functions returns the best combination of parameters, which allows to
# get the best score
def best_params(forest):
  return forest.cv_results_['params'][forest.cv_results_['rank_test_score'][0]-1]

** PERFORMANCE METRIC FUNCTION **

In [ ]:
from sklearn.metrics import r2_score

def performance_metric(y_true, y_predict):
    """ Calculates and returns the performance score between
        true (y_true) and predicted (y_predict) values based on the metric chosen. """

    score = r2_score(y_true, y_predict)

    # Return the score
    return score

### **LINEAR REGRESSION **

In [ ]:
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin

import os

# Create a class to select numerical or categorical columns
# since Scikit-Learn doesn't handle DataFrames yet

class DFSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_names):
        self.feature_names = feature_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.feature_names].values

In [ ]:
# Setting categorical and numerical attributes
cat_features = ["Make", "Model"]
num_features = list(cars_train.drop(cat_features, axis=1))

# Building the Pipelines for categorical and numerical dataframes
numerical_pipeline = Pipeline([
    ("selector", DFSelector(num_features)),
    ("std_scaler", StandardScaler())
])

categorical_pipeline = Pipeline([
    ("selector", DFSelector(cat_features)),
    ("encoder", OneHotEncoder(sparse=True))
])

# full_pipeline
full_pipeline = FeatureUnion(transformer_list =[
    ("num_pipeline", numerical_pipeline),
    ("cat_pipeline", categorical_pipeline)
])


In [ ]:
# Apply the full pipeline
ohe_cars_train = full_pipeline.fit_transform(cars_train) # train set

In [ ]:
from sklearn.linear_model import LinearRegression

sc = createLocalSparkSession().sparkContext

model = LinearRegression()
parameters = {'fit_intercept':[False], 'normalize':[True,False], 'copy_X':[True, False]}

# Spark parallelized GridSearchCV for hyperparameter tuning
gs = GridSearchCV(sc, estimator=model, param_grid=parameters, cv=3, n_jobs=-1, verbose=1, return_train_score=True)
lin_reg = gs.fit(ohe_cars_train, cars_price_train)

In [ ]:
# Best CV parameters
bp = best_params(lin_reg)
best_params(lin_reg)

lin_reg_model = LinearRegression(
                              fit_intercept=bp["fit_intercept"],
                              normalize=bp["normalize"],
                              copy_X=bp["copy_X"])
%time lin_reg_model.fit(ohe_cars_train, cars_price_train)

In [ ]:
# LinearRegression scores for price prediction
ohe_cars_test = full_pipeline.transform(cars_test)

print("Best Linear Regression parameters:")
print(bp)
print("\nLinear Regressor score without CV on train set: %.3f" % lin_reg_model.score(ohe_cars_train, cars_price_train)) #score on train set
print("Linear Regression score without CV on test set: %.3f" % lin_reg_model.score(ohe_cars_test, cars_price_test)) # score on test set
print("Linear Regression Best score with CV=3: %.3f" % best_score(lin_reg, 3)) # -> best score on test set is high

In [ ]:
# Prediction on whole training set
from sklearn.metrics import mean_squared_error

price_predictions_train = lin_reg_model.predict(ohe_cars_train) #using the whole training set for making prediction with the final model given by the best CV parameters

# Reversing np.log operation
price_predictions_train_normal = np.exp(price_predictions_train)
cars_price_train_normal = np.exp(cars_price_train)

# MSE between target values (i.e known) and predicted values
lin_mse = mean_squared_error(cars_price_train_normal, price_predictions_train_normal)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
print(price_predictions_train_normal[580:590])
print('\n')
print(list(cars_price_train_normal[580:590]))

In [ ]:
# Prediction on test set
price_predictions_test = lin_reg_model.predict(ohe_cars_test)

# Reversing np.log operation
price_predictions_test_normal = np.exp(price_predictions_test)
cars_price_test_normal = np.exp(cars_price_test)

# RMSE on test set
final_mse = mean_squared_error(cars_price_test_normal, price_predictions_test_normal)
final_rmse = np.sqrt(final_mse)

final_rmse

In [ ]:
print(price_predictions_test_normal[7650:7660]) #predictions on test set
print('\n')
print(list(cars_price_test_normal[7650:7660])) #known values in test set

In [ ]:
# R2 score
from sklearn.metrics import r2_score
r2_score(cars_price_test_normal, price_predictions_test_normal, multioutput='variance_weighted')

In [ ]:
# Plot predictions
plt.scatter(price_predictions_train_normal, np.exp(cars_price_train), c = "blue", marker = "s", label = "Training data")
plt.scatter(price_predictions_test_normal, np.exp(cars_price_test), c = "lightgreen", marker = "s", label = "Validation data")
plt.title("Linear regression")
plt.xlabel("Predicted values")
plt.ylabel("Real values")
plt.legend(loc = "upper left")
plt.plot([0, 350000], [0, 350000], c = "red")
plt.show()


In [ ]:
# Saving model for type prediction
pickle.dump(lin_reg_model, open("lin_reg_model_final.pkl", "wb"))

In [ ]:
# Loading the model for type prediction
lin_reg_model = pickle.load(open("lin_reg_model_final.pkl", 'rb'))

## **DECISION TREE REGRESSOR**

### COMPLEXITY CURVES

In [ ]:
# Produce complexity curve for varying training set sizes and maximum depths
curves.ModelComplexity_DT(cars_train, cars_price_train)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import make_scorer

def DT_SparkizedGridSearchCV(X, y):
    """ Performs grid search over the 'max_depth' parameter for a
        decision tree regressor trained on the input data [X, y]. """

    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 42)

    # Create a decision tree regressor object
    regressor = DecisionTreeRegressor()

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':[1, 5, 10, 15, 16, 17]}

    # Transform 'performance_metric' into a scoring function using 'make_scorer'
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    sc = createLocalSparkSession().sparkContext
    grid = GridSearchCV(sc, estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    tree_reg = grid.fit(X, y)

    # Return grid search output after fittig the data
    return tree_reg

In [ ]:
from sklearn.model_selection import ShuffleSplit

# Fit the training data to the model using spark parallelized grid search CV
tree_reg = DT_SparkizedGridSearchCV(cars_train, cars_price_train)

# Takign best parameters
bp = best_params(tree_reg)

# Produce the optimal value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(bp['max_depth']))

In [ ]:
"""
Due to the limitation of the spark-sklearn library's implementation of
GridSearchCV, best_estimator_ parameter it's not available, so we need to
fit a DecisionTreeRegressor on the best parameters given to us by gridSearchCV
"""
tree_reg_model = DecisionTreeRegressor(
                              max_depth=bp['max_depth'])
%time tree_reg_model.fit(cars_train, cars_price_train)

In [ ]:
# DecisionTreeRegressor score for price prediction


print("Best Decision Tree Regressor parameters:")
print(bp)
print("\nDecision Tree Regressor score without CV on train set: %.3f" % tree_reg_model.score(cars_train, cars_price_train)) #score on train set
print("Decision Tree Regressor score without CV on test set: %.3f" % tree_reg_model.score(cars_test, cars_price_test)) # score on test set
print("Decision Tree Regressor Best score with CV=10: %.3f" % best_score(tree_reg, 10)) # -> best score on test set is high

In [ ]:
# Prediction on whole training set
price_predictions_train = tree_reg_model.predict(cars_train) #using the whole training set for making prediction with the final model given by the best CV parameters

# Reversing np.log operation
price_predictions_train_normal = np.exp(price_predictions_train)
cars_price_train_normal = np.exp(cars_price_train)

# MSE between target values (i.e known) and predicted values
lin_mse = mean_squared_error(cars_price_train_normal, price_predictions_train_normal)
lin_rmse = np.sqrt(lin_mse)
lin_rmse # is higher than RMSE of linear regression, in fact the best score is smaller (0.58 vs 0.89)

In [ ]:
print(price_predictions_train_normal[1670:1680])
print('\n')
print(list(cars_price_train_normal[1670:1680]))

In [ ]:
# Prediction on test set
price_predictions_test = tree_reg_model.predict(cars_test)

# reversing np.log operation
price_predictions_test_normal = np.exp(price_predictions_test)
cars_price_test_normal = np.exp(cars_price_test)

# RMSE on test set
final_mse = mean_squared_error(cars_price_test_normal, price_predictions_test_normal)
final_rmse = np.sqrt(final_mse)

final_rmse

In [ ]:
print(price_predictions_test_normal[1939:1949]) #predictions on test set
print('\n')
print(list(cars_price_test_normal[1939:1949])) #known values in test set

In [ ]:
# R2 score

from sklearn.metrics import r2_score
r2_score(cars_price_test_normal, price_predictions_test_normal, multioutput='variance_weighted')

In [ ]:
# Saving model for type prediction
pickle.dump(tree_reg_model, open("tree_reg_model_final.pkl", "wb"))

In [ ]:
# Loading the model for type prediction
tree_reg_model = pickle.load(open("tree_reg_model_final.pkl", 'rb'))

## **RANDOM FOREST REGRESSOR**

### COMPLEXITY CURVES

In [ ]:
# Produce complexity curve for varying training set sizes and maximum depths
curves.ModelComplexity_RF(cars_train, cars_price_train)

### Model Training

In [ ]:
# Import 'make_scorer', 'DecisionTreeRegressor', and 'GridSearchCV'
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

def RF_SparkizedGridSearchCV(X, y):
    """ Performs grid search over the 'max_depth' parameter for a
        decision tree regressor trained on the input data [X, y]. """

    # Create cross-validation sets from the training data
    cv_sets = ShuffleSplit(n_splits = 10, test_size = 0.20, random_state = 42)

    # Create a decision tree regressor object
    regressor = RandomForestRegressor()

    # Create a dictionary for the parameter 'max_depth' with a range from 1 to 10
    params = {'max_depth':[16, 17, 18]}

    # Transform 'performance_metric' into a scoring function using 'make_scorer'
    scoring_fnc = make_scorer(performance_metric)

    # Create the grid search cv object --> GridSearchCV()
    sc = createLocalSparkSession().sparkContext
    grid = GridSearchCV(sc, estimator=regressor, param_grid=params, scoring=scoring_fnc, cv=cv_sets)

    # Fit the grid search object to the data to compute the optimal model
    tree_reg = grid.fit(X, y)

    # Return the best parameters after fitting the data
    return tree_reg

In [ ]:
from sklearn.model_selection import ShuffleSplit

# Fit the training data to the model using spark parallelized grid search CV
forest_reg = RF_SparkizedGridSearchCV(cars_train, cars_price_train)

# Takign best parameters
bp = best_params(forest_reg)

# Produce the optimal value for 'max_depth'
print("Parameter 'max_depth' is {} for the optimal model.".format(bp['max_depth']))

In [ ]:
# Fitting the forest

forest_reg_model = RandomForestRegressor(
                              max_depth=bp['max_depth']

)

%time forest_reg_model.fit(cars_train, cars_price_train)

In [ ]:
# RandomForestRegressor score for price prediction

print(bp)
print("\nRandom Forest Regressor score without CV on train set: %.3f" % forest_reg_model.score(cars_train, cars_price_train)) #score on train set
print("Random Forest Regressor score without CV on test set: %.3f" % forest_reg_model.score(cars_test, cars_price_test)) #score on test set
print("Random Forest Regressor Best score with CV=4: %.3f" % best_score(forest_reg, 4)) # -> best score on test set is high

In [ ]:
# Prediction on whole training set
price_predictions_train = forest_reg_model.predict(cars_train) #using the whole training set for making prediction with the final model given by the best CV parameters

# Reversing np.log operation
price_predictions_train_normal = np.exp(price_predictions_train)
cars_price_train_normal = np.exp(cars_price_train)

# MSE between target values (i.e known) and predicted values
lin_mse = mean_squared_error(cars_price_train_normal, price_predictions_train_normal)
lin_rmse = np.sqrt(lin_mse)
lin_rmse

In [ ]:
print(price_predictions_train_normal[25670:25680])
print('\n')
print(list(cars_price_train_normal[25670:25680]))

In [ ]:
# Prediction on test set
price_predictions_test = forest_reg_model.predict(cars_test)

# Reversing np.log operation
price_predictions_test_normal = np.exp(price_predictions_test)
cars_price_test_normal = np.exp(cars_price_test)

final_mse = mean_squared_error(cars_price_test_normal, price_predictions_test_normal)
final_rmse = np.sqrt(final_mse)

final_rmse

In [ ]:
print(price_predictions_test_normal[1870:1880]) #predictions on test set
print('\n')
print(list(cars_price_test_normal[1870:1880])) #known values in test set

In [ ]:
# r2 score between hold out prices and predicted prices
r2_score(cars_price_test_normal, price_predictions_test_normal, multioutput='variance_weighted')

In [ ]:
# Saving model for type prediction
pickle.dump(forest_reg_model, open("forest_reg_model_final.pkl", "wb"))

In [ ]:
# Loading the model for type prediction
forest_reg_model = pickle.load(open("forest_reg_model_final.pkl", 'rb'))

## **CROSS VALIDATION**

In [ ]:
def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

### LINEAR REGRESSION

In [ ]:
# Cross val score on training set, although we already used GridSearchCV

train_scores = cross_val_score(lin_reg_model, ohe_cars_train, np.exp(cars_price_train),
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-train_scores)

display_scores(tree_rmse_scores)

In [ ]:
train_scores = cross_val_score(lin_reg_model, ohe_cars_train, np.exp(cars_price_train),
                          cv=10)

display_scores(train_scores)

### DECISION TREE

In [ ]:
# Cross val score on training set

train_scores = cross_val_score(tree_reg_model, cars_train, np.exp(cars_price_train),
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-train_scores)

display_scores(tree_rmse_scores)

In [ ]:
train_scores = cross_val_score(tree_reg_model, cars_train, np.exp(cars_price_train),
                          cv=10)

display_scores(train_scores)

###RANDOM FOREST

In [ ]:
from sklearn.model_selection import KFold

# Cross val score on training set, although we already used grid search CV
train_scores = cross_val_score(forest_reg_model, cars_train, np.exp(cars_price_train),
                         scoring="neg_mean_squared_error", cv=KFold(10, shuffle=True))
forest_rmse_scores = np.sqrt(-train_scores)

display_scores(forest_rmse_scores)

In [ ]:
train_scores = cross_val_score(forest_reg_model, cars_train, np.exp(cars_price_train),
                          cv=5)

display_scores(train_scores)

## **PREDICTIONS ON FINAL MODEL AND CONCLUSIONS**

In [ ]:
final_model = forest_reg_model

**FEATURE IMPORTANCE**

In [ ]:
# Feature importance computed with cross validation
from rfpimp import cv_importances, plot_importances
%time I = cv_importances(final_model, cars_train, np.exp(cars_price_train), k=5)
plot_importances(I)

In [ ]:
# Prediction on the hold out test set
final_predictions = final_model.predict(cars_test)

final_mse = mean_squared_error(np.exp(cars_price_test), np.exp(final_predictions))

final_rmse = np.sqrt(final_mse)

In [ ]:
# RMSE and scores
print("Test RMSE: %f " % final_rmse)
print("Score on held-out Test Set: %f " % final_model.score(cars_test, cars_price_test))
print("R2 Score: %f" % r2_score(np.exp(cars_price_test), np.exp(final_predictions)))

In [ ]:
# Cross validation on the entire dataset, since we are good with our final model

features = cleaned_cars.drop(['Price'], axis=1)
prices = cleaned_cars['Price'].copy()

final_rmses= cross_val_score(final_model, features, np.exp(prices),
                          scoring="neg_mean_squared_error", cv=KFold(10, shuffle=True))

final_rmse_scores = np.sqrt(-final_rmses)
display_scores(final_rmse_scores)

In [ ]:
# Final cross validation scores in percentage, computed on the whole dataset
final_scores = cross_val_score(tree_reg_model, features, np.exp(prices),
                          cv=KFold(10, shuffle=True))

display_scores(final_scores)